### PyTorch

Facultad de Ingeniería - Universidad de la República - Uruguay

Setiembre de 2025

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pln-fing-udelar/cursos/blob/master/tutoriales/PyTorch-Intro.ipynb)


En este notebook presentaremos una breve introducción a las características principales de PyTorch, una biblioteca para deep learning en Python. Este módulo asume que el lector conoce NumPY, la biblioteca de análisis numérico básico en Python, y scikit-learn, la biblioteca de aprendizaje automático más popular en el lenguaje (la utilizaremos para funciones auxiliares de carga de datos y alguna tarea más, pero no para el aprendizaje propiamente dicho).


Referencias:

- El notebook está basado principalmente en el libro [Machine Learning with PyTorch and Scikit-Learn](https://github.com/rasbt/machine-learning-book), de Sebastian Raschka, Yuxi Liu y Vahid Mirjalili, en particular los capítulos 12 y 13
- La documentación de PyTorch tienen un [tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html) mostrando cómo entrenar una red neuronal que clasifica prendas, entrenado sobre el dataset FashionMNIST
- Hay una muy breve [introducción](https://docs.pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html) a autograd en la documentación de PyTorch.
- Para una introducción paso a paso al uso de PyTorch para DL, el curso de Sebastian Raschka [Deep Learning fundamentals](https://lightning.ai/courses/deep-learning-fundamentals/) es un material excelente

In [1]:
import torch
import numpy as np

# Este notebook fue elaborado con la versión 2.0.1 de Torch y la versión 1.24.3 de NumPy
print("Versión de Torch:",torch.__version__)
print("Versión de Numpy:",np.__version__)

Versión de Torch: 2.2.2+cpu
Versión de Numpy: 1.24.3


## 1. Tensores

Las estructuras básicas de PyTorch son los _tensores_. Los tensores son generalizaciones de los escalares, vectores, matrices, etc. Un escalar es un tensor de rango 0, un vector es un tensor de rango 1, y una matriz es un tensor de rango 2. Los tensores, sin embargo, pueden tener rangos arbitrariamente grandes.  Los tensores son parecidos a los arrays de PyTorch, pero están optimizados para las operaciones de diferenciación automática y pueden correr en GPUs (volveremos sobre esto). Los elementos de un tensor (igual que como sucede con los arrays de NumPy y a diferencia de las listas de Python) tienen un tipo de datos únicos. Es importante tener claro el tipo (y es una buena práctica especificarlo al crear el tensor), ya que puede hacer una diferencia importante cuando se manejan volúmenes grandes de datos (por ejemplo, un elemento de tipo `torch.float64` ocupará el doble de espacio que uno de tipo `torch.float32`


Los tensores pueden crearse a partir de listas de Python o arrays de NumPy:

In [2]:
t_a = torch.tensor([1,2,3], dtype=torch.int32)
print(t_a)

b = np.array([3,4,5], dtype=np.float32)
t_b = torch.from_numpy(b)
print(t_b)

tensor([1, 2, 3], dtype=torch.int32)
tensor([3., 4., 5.])


Podemos consultar las propiedades de un tensor, tales como tipo de datos, y su forma:

In [3]:
t_c = torch.rand(3,4) # El tipo por defecto será torch.float32
print(t_c)

# El tensor tiene tres filas y 4 columnas
print(t_c.shape)
print(t_c.dtype)

tensor([[0.4426, 0.3929, 0.2000, 0.7114],
        [0.9016, 0.0849, 0.1904, 0.7641],
        [0.9752, 0.6045, 0.4169, 0.2851]])
torch.Size([3, 4])
torch.float32


Los tensores pueden dinámicamente cambiar su forma (i.e. cuántos valores tiene cada dimensión). PyTorch resuelve de forma "inteligente" estos cambios, siempre que se mantenga la cantidad de elementos. El siguiente ejemplo utiliza el método `reshape` para  transformar un tensor que es un vector de 10 elementos, en una matriz de 2 filas por 5 columnas (existe un método similar, `view`, que cumple la misma función, y que sólo difiere en que exige que los datos estén en bloques contiguos de memoria).

In [4]:
t_d = torch.ones(10)
t_d1 = t_d.reshape(2,5) # La convención para el orden es, igual que en numpy, fila/columna
print(t_d)
print(t_d1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


Si una de las dimensiones de reshape vale -1, entonces PyTorch resuelve de forma inteligente el valor, para que el total de elementos sea el adecuado

In [5]:
t_d2 = t_d.reshape(5,-1) # 5 filas, las columnas que sean necesarias
print(t_d2)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])


Existe un atributo muy relevante para los tensores, `device`. Este atributo indica en qué dispositivo se almacena (y por lo tanto, se procesa).



In [6]:
print("Dispositivo actual: {}".format(t_a.device))

Dispositivo actual: cpu


Si tenemos disponible una GPU, podemos decirle a PyTorch que el tensor la utilice, simplemente modificando el valor del atributo `device`. Debemos previamente verificar que es posible, porque de lo contrario nos dará un error.

In [7]:
# Esto funciona porque este notebook está corriendo en una GPU
if torch.cuda.is_available():
  t_a = t_a.to(torch.device("cuda"))

print(t_a.device)

cpu


Usualmente, lo que hacemos es, al comienzo del notebook, chequear si hay disponible una CPU y asignar a los tensores una GPU si existe, o una cpu, en caso contrario:

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU (T4) is available and will be used.")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}") # Get the name of the first GPU
else:
    device = torch.device("cpu")
    print("GPU (T4) is not available, using CPU instead.")

t = torch.tensor([1,1,1], device=device)
print (t.device)

GPU (T4) is not available, using CPU instead.
cpu


## 2. Operaciones con Tensores

Igual que numpy permite operar con arrays, PyTorch utiliza prácticamente las mismas reglas y operaciones para realizar operaciones matemáticas sobre los tensores.

El siguiente ejemplo multiplica elemento a elemento dos tensores de tamaño 2x3 (el primero, creado a partir de números aleatorios con una distribución uniforme en \[0,1), y el segundo con una distribución normal de media 0 y desviación estándar 1.

In [9]:
torch.manual_seed(10) # Es buena costumbre fijar la semilla para que los resultados sean reproducibles
t1 = torch.rand(5,2)
t2 = torch.normal(mean=0,std=1,size=(5,2))

print(t1)
print(t2)

print(torch.multiply(t1,t2))

tensor([[0.4581, 0.4829],
        [0.3125, 0.6150],
        [0.2139, 0.4118],
        [0.6938, 0.9693],
        [0.6178, 0.3304]])
tensor([[-1.0531,  0.9457],
        [-0.8307, -0.3713],
        [ 1.2137, -0.6678],
        [-0.4038, -0.1300],
        [ 0.7410, -0.1425]])
tensor([[-0.4824,  0.4567],
        [-0.2596, -0.2284],
        [ 0.2597, -0.2750],
        [-0.2802, -0.1260],
        [ 0.4578, -0.0471]])


Podemos también hacer cálculos sobre ciertas axes (axis=0 son las filas, axis=1 las columnas, y así sucesivamente). Por ejamplo, para calcular la media de cada una de las columnas del segundo tensor con números aleatorios, utilizamos `torch.mean`:

In [10]:
print(torch.mean(t2,axis=0)) # Debería dar algo cercano a 0...

tensor([-0.0666, -0.0732])


Para calcular multiplicaciones entre matrices, se utiliza `torch.matmul`, que realiza las operaciones [usuales](https://es.wikipedia.org/wiki/Multiplicaci%C3%B3n_de_matrices) (también puede utilizarse el operador `@` para lograr exactamente el mismo resultado).

In [11]:
t4 = torch.tensor([[1,2],[3,4]])
t5 = torch.tensor([[5,6],[7,8]])

print(torch.matmul(t4,t5))

tensor([[19, 22],
        [43, 50]])


Si queremos calcular la [norma](https://es.wikipedia.org/wiki/Norma_vectorial) de un tensor, utilizaremos `torch.linalg.norm`:

In [12]:
t6 = torch.rand(100)
print(torch.linalg.norm(t6,ord=2))
print(torch.linalg.norm(t6,ord=1))


tensor(5.9523)
tensor(52.4014)


## 3. Carga de datos

Habiendo visto las operaciones básicas sobre tensores, veamos cómo entrenar una red neuronal utilizando PyTorch. Antes vamos a describir algunas funciones útiles que PyTorch provee para manejar datasets, especialmente en aquellos casos donde no podemos guardar todo el dataset en memoria, y deberemos irlo recuperando en pedazos (o batchs) desde disco, así como realizar otras funciones de preprocesamiento antes del aprendizaje propiamente dicho.

Para mantener separado el código que procesa los datos del que realiza el entrenamiento, existen dos primitivas fundamentales: `torch.utils.data.Dataset` que permite manejar las instancias de datos y sus correspondientes etiquetas, y `torch.utils.data.DataLoader` que permite construir un interable a partir de un Dataset. El tipo de Dataset más básico es simplemente un tensor, y podemos crear un DataLoader a partir de él fácilmente:

In [13]:
from torch.utils.data import DataLoader
t = torch.rand(20)
data_loader = DataLoader(t)

Habiendo construido el dataset, podemos iterar fácilmente sobre él:

In [14]:
for item in data_loader:
    print(item)

tensor([0.4385])
tensor([0.5048])
tensor([0.3810])
tensor([0.4269])
tensor([0.1977])
tensor([0.1699])
tensor([0.6641])
tensor([0.9510])
tensor([0.1006])
tensor([0.0280])
tensor([0.2296])
tensor([0.9799])
tensor([0.9500])
tensor([0.0135])
tensor([0.6213])
tensor([0.5674])
tensor([0.9417])
tensor([0.3501])
tensor([0.6649])
tensor([0.2524])


Podemos especificar que los elementos se procesen en batchs de cierto tamaño:

In [15]:
data_loader = DataLoader(t,batch_size=6,drop_last=False) # No descartamos el último batch si está incompleto
for i, batch in enumerate(data_loader,1):
    print(f'batch {i}:', batch)

batch 1: tensor([0.4385, 0.5048, 0.3810, 0.4269, 0.1977, 0.1699])
batch 2: tensor([0.6641, 0.9510, 0.1006, 0.0280, 0.2296, 0.9799])
batch 3: tensor([0.9500, 0.0135, 0.6213, 0.5674, 0.9417, 0.3501])
batch 4: tensor([0.6649, 0.2524])


Es muy común que tengamos dos tensores que nos interese procesar al mismo tiempo. El caso típico es un tensor con las features, y otro con las etiquetas. Para eso, la clase `TensorDataset` (que es una tipo especial de `torch.utils.data.Dataset`, que PyTorch permite definir de forma genérica) nos permite manejarlos como una unidad:

In [16]:
from torch.utils.data import TensorDataset

# Creamos un mini dataset que tiene 4 elementos y 3 atributos por cada uno
t_x = torch.rand([4,3],dtype=torch.float32)
t_y = torch.arange(4)
joint_dataset = TensorDataset(t_x,t_y)

for example in joint_dataset:
    print('x:', example[0], ' y:', example[1])

x: tensor([0.3896, 0.0956, 0.7927])  y: tensor(0)
x: tensor([0.8453, 0.8823, 0.5649])  y: tensor(1)
x: tensor([0.0279, 0.1002, 0.4475])  y: tensor(2)
x: tensor([0.4481, 0.0462, 0.8099])  y: tensor(3)


Con este dataset, podemos, por supuesto, crear un nuevo DataLoader. En este caso, vamos a especificarle, además, que cada vez que recorra el dataset, mezcle las instancias de forma aleatoria. Esto será muy útil cuando recorramos varias veces el conjunto de entrenamiento para entrenar la red neuronal. El método `manual_seed` permite especificar una semilla para el generador de datos aleatorios, parque los resultados puedan ser reproducibles.

In [17]:
torch.manual_seed(10)
data_loader = DataLoader(dataset=joint_dataset, batch_size=2, shuffle=True, drop_last=False)

Realizamos una iteración...

In [18]:
for i, batch in enumerate(data_loader,1):
    print(f'batch {i}:', batch)

batch 1: [tensor([[0.3896, 0.0956, 0.7927],
        [0.0279, 0.1002, 0.4475]]), tensor([0, 2])]
batch 2: [tensor([[0.8453, 0.8823, 0.5649],
        [0.4481, 0.0462, 0.8099]]), tensor([1, 3])]


Y luego otra, sobre el mismo dataset...

In [19]:
for i, batch in enumerate(data_loader,1):
    print(f'batch {i}:', batch)

batch 1: [tensor([[0.4481, 0.0462, 0.8099],
        [0.0279, 0.1002, 0.4475]]), tensor([3, 2])]
batch 2: [tensor([[0.8453, 0.8823, 0.5649],
        [0.3896, 0.0956, 0.7927]]), tensor([1, 0])]


## 4. Cargando datos con torchvision

Los métodos de `Dataset` y `DataLoader` permite construir datasets de forma muy general, a partir de nuestros datos, utilizando PyTorch. Sin embargo, para los datasets más populares, existen bibliotecas como `TorchVision` o `TorchText` que ya los proveen. Veamos, por ejemplo, cómo importar el dataset [MNIST](http://yann.lecun.com/exdb/mnist/) de números escritos a mano, utilizando `torchvision`

In [20]:
import torchvision
from torchvision import transforms

image_path = './'

# Esto nos permite convertir las features de los píxeles en tensores y los normaliza al rango [0,1]
transform = transforms.Compose([transforms.ToTensor()])

# Importamos los datasets
# Las etiquetas son valores entre 0 y 9 para representar los dígitos.
mnist_train_dataset = torchvision.datasets.MNIST(root=image_path, train=True, transform=transform, download=True)
mnist_test_dataset  = torchvision.datasets.MNIST(root=image_path, train=False, transform=transform, download=True)

batch_size = 64
torch.manual_seed(10)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:01<00:00, 7502405.19it/s]


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 218031.12it/s]


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 1933182.67it/s]


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|███████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 1368670.79it/s]

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



## 5. El modelo de computación de PyTorch y autograd

PyTorch permite construir _grafos de computación_ para derivar las relaciones entre los tensores, cuando se realizan operaciones entre ellos (como veremos más adelante, una red neuronal no es más que una serie de cálculos sobre tensores, partiendo del tensor inicial con la entrada).

Empecemos con un ejemplo bien sencillo, que utiliza solamente escalares (i.e. tensores de dimensión 0). Supongamos que $a$, $b$ y $c$ son tensores y que  $ z = 2(a^2-b^3) +c$

In [21]:
a=torch.tensor(2.5)
b=torch.tensor(3.2)
c=torch.tensor(4)

# z es simplemente la operación aritmética "clasica" entre números reales.
# Al asignar z, hacemos el cálculo (es lo que llamamos pasada forward por el grafo de computación
z= 2*(a**2-b**3)+c
print(z)

tensor(-49.0360)


Hasta aquí, nada que no podamos hacer con NumPy (de hecho... con Python). Pero PyTorch incorpora la capacidad de diferenciación automática (a través de una biblioteca llamada `autograd`), los que nos permite llevar registro de las derivadas del resultado respecto de los parámetros involucrados (al vector con estas derivadas se le llama _gradiente_, y es clave para minimizar la pérdida al entrenar redes, como veremos más adelante). Intentemos hacer lo mismo que antes, pero ahora indicamos a PyTorch que queremos calcular el gradiente de z respecto a a, b y c.

In [22]:
a=torch.tensor(2.5, requires_grad=True)
b=torch.tensor(3.2, requires_grad=True)
# Indicamos que c es un tensor de reales, porque solamente se puede calcular el gradiente
# de números reales o complejos
c=torch.tensor(4., requires_grad=True)

z= 2*(a**2-b**3)+c
print(z)

tensor(-49.0360, grad_fn=<AddBackward0>)


El tensor z ahora "sabe" cómo fue calculado, porque autograd lleva el registro. Con esto podemos utilizar el método `backward` sobre z, para obtener el gradiente, utilizando backpropagation. Hagamos la prueba:

In [23]:
# Aquí está toda la magia
# Utiliza backpropagation para calcular el gradiente del resultado respecto a cada uno de los
# parámetros por los que estamos controlando
z.backward()
print(a.grad)
print(b.grad)
print(c.grad)

tensor(10.)
tensor(-61.4400)
tensor(1.)


(Es interesante notar que podemos llamar una sola vez a este método, porque, para ahorrar memoria, PyTorch "pierde" el grafo de cómputo al finalizar el proceso de cálculo, salvo que especifiquemos `retain_graph=True` al invocarlo.

Como esta es una función muy sencilla, podemos calcular directamente las derivadas y verificar los cálculos.


\begin{equation}
\frac{\partial z}{\partial a} = 4a = 10 \\
\frac{\partial z}{\partial b} = -6b = -61.44 \\
\frac{\partial z}{\partial c} =1
\end{equation}

Veamos un nuevo caso, ahora un poco más parecido al entrenamiento una red neuronal, pero que utiliza exactamente los mismos principios.

Supongamos que tenemos una red con una sola neurona, lineal, que toma dos valores $x_1$ y $x_2$ de entrada, lo multiplica por los respectivos pesos $w_1$ y $w_2$ y le suma un sesgo $b$

In [24]:
# Nuestros parámetros son w y b
# Los inicializamos con valores arbitrarios
w = torch.tensor([0.5,0.3], requires_grad = True)
b = torch.tensor([1.0], requires_grad = True)

# Nuestra entrada es x, que representaremos por un tensor, al que no le calcularemos gradiente
x = torch.tensor([1.4,2.1])

# Calculamos z
# Verificar que 2.33 = 0.5*1.4 + 0.3*2.1 + 1
z = torch.dot(w,x) + b
print(z)

tensor([2.3300], grad_fn=<AddBackward0>)


Además, supongamos que tenemos una función de pérdida que compara el resultado con un objetivo conocido y cálcula su pérdida cuadrática

In [25]:
y = torch.tensor(1.1)
loss = (y -z).pow(2).sum()
print(loss)

tensor(1.5129, grad_fn=<SumBackward0>)


Y ahora, la magia de backpropagation...

In [26]:
loss.backward()

In [27]:
print('dL/dw:',w.grad)
print('dL/db:',b.grad)

dL/dw: tensor([3.4440, 5.1660])
dL/db: tensor([2.4600])


Podemos ver que el atributo `grad` de $w$ y de $b$ contiene el gradiente de la función de pérdida. Verificamos nuevamente (y por última vez...) que PyTorch hace bien las cuentas (lo haremos para el caso de $w_1$)

$$ \frac{\partial L}{\partial w_1} = \frac{\partial (z-y)^2}{\partial w_1} = \frac{\partial (z-y)^2}{\partial z}\cdot \frac{\partial z}{\partial w_1} = 2(z-y)\cdot \frac{x_1w_1 + x_2w_2 +b}{\partial w_1} = 2(z-y)x_1 = 2(2.33-1.1)\times1.4 = 3.444$$



## 6. nn.Sequential

PyTorch provee una clase [`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) que permite especificar redes feed forward de forma muy sencilla. Veamos un ejemplo:



In [28]:
from torch import nn
model = nn.Sequential(
    nn.Linear(4,16),
    nn.ReLU(),
    nn.Linear(16,32),
    nn.ReLU()
)

model

Sequential(
  (0): Linear(in_features=4, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=32, bias=True)
  (3): ReLU()
)

La red anterior recibe 4 reales como entrada y tiene dos capas: la primera tiene 16 nodos, y su función de activación es una ReLU, mientras que la segunda tiene 32 nodos y también una ReLU como salida. Por lo tanto, esta red tendrá como salida un valor real (y por lo tanto sería adecuada para un problema de regresión). Sobre este modelo, es posible especificar diferentes funciones de activación, aplicar regularizaciones, entre otras muchas funciones.

## 8. Definición y entrenamiento de una red neuronal

A partir de los datos de los dígitos de MNIST que importamos previamente, vamos a intentar predecir con una red neuronal las clases correspondientes.

Primero, construimos el modelo de red (seguimos el ejemplo del libro de Rashka mencionado en la introducción):

In [29]:
hidden_units = [32,16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size [2]

# Nuestro primer paso es convertir las imágenes  a un vector
all_layers = [nn.Flatten()]

for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1],10))

model = nn.Sequential(*all_layers)

print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)


A continuación, realizamos el entrenamiento. Para esto, especificamos la función de pérdida (en esta caso, Entropía cruzada) y el optimizador (Adam). Nos interesa usar entropía cruzada porque es la función de pérdida típica para softmax. El optimizador es quien se encarga de actualizar los parámetros a partir de los valores calculados del gradiente (el caso más conocido es descenso por gradiente).



In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(10)


Entrenamos por 20 épocas...

In [31]:
num_epochs = 20
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_dl:
        # Paso forward, obtenemos la predicción de la red con los valores actuales
        pred = model(x_batch)
        # Calculamos la pérdida.
        loss = loss_fn(pred,y_batch)

        # Backpropagation!
        loss.backward()

        # Ajustamos los pesos utilizando el algoritmo de optimización, en este caso Adam
        optimizer.step()

        # Ponemos los gradientes en zero
        # Generalmente, los optimizadores mantienen el gradiente de la pasada anterior
        optimizer.zero_grad()

        is_correct = (torch.argmax(pred,dim=1) == y_batch).float()

        accuracy_hist_train += is_correct.sum()

    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch}  Accuracy '
          f'{accuracy_hist_train:.4f}')




Epoch 0  Accuracy 0.8467
Epoch 1  Accuracy 0.9297
Epoch 2  Accuracy 0.9455
Epoch 3  Accuracy 0.9534
Epoch 4  Accuracy 0.9586
Epoch 5  Accuracy 0.9633
Epoch 6  Accuracy 0.9658
Epoch 7  Accuracy 0.9684
Epoch 8  Accuracy 0.9697
Epoch 9  Accuracy 0.9731
Epoch 10  Accuracy 0.9741
Epoch 11  Accuracy 0.9755
Epoch 12  Accuracy 0.9762
Epoch 13  Accuracy 0.9779
Epoch 14  Accuracy 0.9789
Epoch 15  Accuracy 0.9799
Epoch 16  Accuracy 0.9807
Epoch 17  Accuracy 0.9817
Epoch 18  Accuracy 0.9826
Epoch 19  Accuracy 0.9836


Nuestro modelo logró una accuracy de 98.36% sobre el conjunto de entrenamiento. Veamos ahora cómo es su performance sobre el conjunto de evaluación, para verificar que no hubo sobreajuste...

In [32]:
print((mnist_test_dataset.data).shape)

torch.Size([10000, 28, 28])


In [33]:
# Predecimos utilizando el modelo

# Antes normalizamos los valores RGB
# Esto lo hago porque no paso por el transform
pred = model(mnist_test_dataset.data/255.)

is_correct = ( torch.argmax(pred,dim=1) == mnist_test_dataset.targets).float()
print(f'Test accuracy: {is_correct.mean():.4f}')

Test accuracy: 0.9674


## 9. Más para leer

En el capítulo 13 del libro explica (y hay código asociado) cómo utilizar la biblioteca Lightining Trainer y TensorBoard para mostrar mejor los resultados. El código está disponible en [github](https://github.com/rasbt/machine-learning-book/tree/main/ch13)